Utilities:

In [1]:
import sys
sys.path.append('..')
from shared.aocutil import slice_deque, update_deque


Part 1

In [2]:
from copy import deepcopy
from collections import deque

monkeys = []
monkey_biz = []
for l in open('../inputs/day11-input'):
    if l == '\n':
        monkeys.append(deepcopy(monkey_biz))
        monkey_biz.clear()
    else:
        monkey_biz.append(l.strip())
monkeys.append(monkey_biz.copy())
del monkey_biz

# Clean up data; a more polite version of me would make a Monkey class with named attributes
for i, biz in enumerate(monkeys):
    biz = biz[1:]  # Remove the first line; already encoded in 'monkeys' via index
    biz[0] = deque([int(_) for _ in biz[0].split(': ')[1].split(', ')])  # Extract item list
    biz[1] = tuple(biz[1][21:].split())  # Extract operator and operand
    biz[2] = int(biz[2][19:]) # Extract dividend for floor operation
    biz[3] = int(biz[3][25:]) # Extract monkey index for 'true' condition
    biz[4] = int(biz[4][25:]) # Extract monkey index for 'false' condition
    biz.append(0)  # Add an entry for number of items inspected by this monkey
    monkeys[i] = biz


In [4]:
"""
One "round":
0. For each monkey:
    1. For each starting value:
        2. Monkey.inspect ("operation" on next in FIFO)
        3. Me.relief (value // 3)
        4. Monkey.test ("test")
        5. Monkey.throw(bool)
        6. OtherMonkey.catch (append to FIFO)

Over 20 rounds, count #2 for each monkey. Multiply the top 2 values to get "monkey business".
"""


for _ in range(20):
    # One round:
    for m, biz in enumerate(monkeys):
        for instr in range(len(biz[0])):  # Step through the list of instructions
            try:
                item_val = biz[0].popleft()
                biz[5] += 1
            except IndexError as e:
                print(f'Monkey {m}, instruction {instr}: {e}')
                continue

            operand = biz[1][1]
            match biz[1][0]:
                case '+':
                    item_val += int(operand)
                case '-':
                    item_val -= int(operand)
                case '*':
                    if operand == 'old':
                        item_val = item_val ** 2
                    else:
                        item_val *= int(operand)
                case '/':
                    item_val /= int(operand)

            item_val = item_val // 3
            throw_to = biz[3] if not item_val % biz[2] else biz[4]
            monkeys[throw_to][0].append(item_val)

inspections = [biz[5] for biz in monkeys]
print(inspections)
print(f'Solution: 342 * 352 = 120384')
# If you don't see those values in the list, you probably didn't restart the iPython kernel to flush the list-of-lists from memory first.

[39, 689, 702, 95, 573, 694, 703, 26]
Solution: 342 * 352 = 120384
